<a href="https://colab.research.google.com/github/naitotomoyuki/YoloLearning_with_ggl_colab/blob/main/yolo11n_Shikibo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
import os

In [2]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [3]:
!export LC_ALL=C.UTF-8
!export LANG=C.UTF-8
!export LANGUAGE=C.UTF-8

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!pip install -U torch torchvision torchaudio
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.0 MB/s eta 0:00:00


In [6]:
#!rsync -ah --progress /content/drive/MyDrive/dataset/ /content/dataset/
#超重要！！！！！
#ローカルPCでdatasetをdataset.zipに圧縮してMyDriveへコピーしてください。

# ZIPファイルをGoogle DriveからColabへコピー
!rsync -ah --progress /content/drive/MyDrive/dataset.zip /content/

# ZIPファイルを解凍
!unzip -o /content/dataset.zip -d /content/

# (任意) 解凍後のZIPファイルを削除してディスクを節約
!rm /content/dataset.zip

sending incremental file list
dataset.zip
        403.16M 100%   81.01MB/s    0:00:04 (xfr#1, to-chk=0/1)
Archive:  /content/dataset.zip
   creating: /content/dataset/
  inflating: /content/dataset/data.yaml  
   creating: /content/dataset/images/
  inflating: /content/dataset/images/CAM1_250804080357.jpg  
  inflating: /content/dataset/images/CAM1_250805024725.jpg  
  inflating: /content/dataset/images/CAM1_250805050017.jpg  
  inflating: /content/dataset/images/CAM1_250805151927.jpg  
  inflating: /content/dataset/images/CAM1_250806104530.jpg  
  inflating: /content/dataset/images/CAM1_250807090505.jpg  
  inflating: /content/dataset/images/CAM1_250808152706.jpg  
  inflating: /content/dataset/images/CAM2_250804080458.jpg  
  inflating: /content/dataset/images/CAM2_250805091658.jpg  
  inflating: /content/dataset/images/CAM2_250805132757.jpg  
  inflating: /content/dataset/images/CAM2_250805140230.jpg  
  inflating: /content/dataset/images/CAM2_250805140521.jpg  
  inflating: /conten

In [7]:
model_path = "/content/drive/MyDrive/yolo11n.pt"
# YAMLファイルのパス
data_yaml = "/content/dataset/data.yaml"

In [8]:
import yaml

# YAMLファイルを読み込む
with open(data_yaml, "r") as file:
    data = yaml.safe_load(file)

# 変更したいパス
data["test"] = "./images/test"
data["train"] = "./images/train"
data["val"] = "./images/val"

# YAMLファイルを書き戻す
with open(data_yaml, "w") as file:
    yaml.safe_dump(data, file, default_flow_style=False)

print("data.yaml のパスを更新しました。")

data.yaml のパスを更新しました。


In [9]:
import numpy as np
import os
from ultralytics import YOLO

def train_yolo(model_path, data_yaml, epochs, batch_size, img_size, output_dir, iou_threshold, mosaic, mixup):
    """
    Train YOLO11n model with enhanced settings for small overlapping object detection.
    """
    print(f"Loading model from {model_path}")
    model = YOLO(model_path)

    # Train the model with optimized settings
    print(f"Training YOLO11 with {epochs} epochs...")
    model.train(
        data=data_yaml,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project=output_dir,
        name="yolo11_training",
        iou=iou_threshold,  # Adjust IoU threshold for better object separation
        mosaic=mosaic,  # Enhance data augmentation with mosaic images
        mixup=mixup,  # Mixup augmentation for diverse training
        degrees=4,  # Rotation augmentation
    )

    print(f"Training complete. Results saved to {output_dir}")

# Colab 用にパラメータを手動で指定
#model_path = "/content/drive/MyDrive/yolo12s.pt"  # モデルのパス (Google Drive に保存している場合は適宜変更)
#data_yaml = "/content/drive/MyDrive/MaiNDLab/dataset/data.yaml"  # データセットの YAML ファイル
epochs = 300  # エポック数
batch_size = 64  # バッチサイズ (Reduced to mitigate OOM error)
img_size = 640  # 画像サイズ
iou_threshold = 0.9  # IoU 閾値
mosaic = 0.1  # モザイクの確率
mixup = 0.0  # Mixup の確率
output_dir = "runs_experiment"  # 出力ディレクトリ

# 出力ディレクトリを作成
os.makedirs(output_dir, exist_ok=True)

# YOLO の学習実行
train_yolo(
    model_path=model_path,
    data_yaml=data_yaml,
    epochs=epochs,
    batch_size=batch_size,
    img_size=img_size,
    output_dir=output_dir,
    iou_threshold=iou_threshold,
    mosaic=mosaic,
    mixup=mixup
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading model from /content/drive/MyDrive/yolo11n.pt
Training YOLO11 with 300 epochs...
Ultralytics 8.3.182 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset/data.yaml, degrees=90, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=300, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, ha

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

YOLO11n summary: 181 layers, 2,591,010 parameters, 2,590,994 gradients, 6.4 GFLOPs

Transferred 448/499 items from pretrained weights
Freezing layer 'model.23.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 4700.0±1205.5 MB/s, size: 1553.1 KB)


train: Scanning /content/dataset/labels/train... 184 images, 4 backgrounds, 1 corrupt: 100%|██████████| 184/184 [00:00<00:00, 1530.92it/s]

...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    300/300      13.1G      1.743      1.954      1.774         57        640: 100%|██████████| 3/3 [00:00<00:00,  4.89it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.56it/s]


                   all         51         54      0.534      0.101     0.0679     0.0239

300 epochs completed in 0.109 hours.
Optimizer stripped from runs_experiment/yolo11_training/weights/last.pt, 5.5MB
Optimizer stripped from runs_experiment/yolo11_training/weights/best.pt, 5.5MB

Validating runs_experiment/yolo11_training/weights/best.pt...
Ultralytics 8.3.182 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40507MiB)
YOLO11n summary (fused): 100 layers, 2,583,322 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.23it/s]


                   all         51         54      0.528      0.113     0.0861     0.0275
               Kakou10          9          9      0.189      0.222      0.154     0.0205
                Kakou3         22         25       0.13       0.12      0.121     0.0649
                 Ori10          1          1          1          0          0          0
                  Ori3         18         18       0.32      0.222      0.156     0.0519
                  Ori5          1          1          1          0          0          0
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs_experiment/yolo11_training
Training complete. Results saved to runs_experiment


In [10]:
# best.pt のパス
best_model_path = 'runs_experiment/yolo11_training/weights/best.pt'

# best.pt が存在すれば自動でダウンロード
if os.path.exists(best_model_path):
    files.download(best_model_path)
else:
    print("best.pt が見つかりませんでした。")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>